In [1]:
from langgraph.graph import StateGraph
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [2]:
load_dotenv()

True

In [3]:
# Get the key from env variable
groq_key = os.getenv("GROQ_API_KEY")

In [4]:
print("Loaded key:", groq_key is not None)  # Debug: should print Tru

Loaded key: True


In [5]:
model = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)